In [3]:
from __future__ import division
import tarfile
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
data = pd.read_csv("spam.csv",encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
y = data['v1'].as_matrix()
X_text = data['v2'].as_matrix() 
print(X_text.shape)
print(y.shape)

(5572,)
(5572,)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
sw = stopwords.words("english")
cv = CountVectorizer(stop_words =sw)
tcv = cv.fit_transform(X_text).toarray()
#print(cv.vocabulary_)
print(tcv.shape)
print(y.shape)

(5572, 8536)
(5572,)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=sw,lowercase=True)
X = vectorizer.fit_transform(X_text).toarray()
print(X.shape)
print(y.shape)

(5572, 8536)
(5572,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4457, 8536)
(4457,)
(1115, 8536)
(1115,)


In [9]:
y_train

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression()
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy_score(y_test,pred)

0.9443946188340807

In [9]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy_score(y_test,pred)

0.885201793721973

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(500,500))
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy_score(y_test,pred)

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma=0.1,C=1,kernel='rbf')
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy_score(y_test,pred)

In [12]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
y_train_oh=tf.one_hot(tf.cast(y_train,tf.int32),2)
y_tr=sess.run(y_train_oh)
y_test_oh=tf.one_hot(tf.cast(y_test,tf.int32),2)
y_te=sess.run(y_test_oh)
#print(y_tr.shape)
#print(y_train,y_tr)

In [13]:
num_hidden_nodes1 = 2000
num_hidden_nodes2 = 1000
num_hidden_nodes3 = 256
keep_prob = 0.5
# numFeatures = the number of words extracted from each email
numFeatures = X_train.shape[1]
# numLabels = number of classes we are predicting (here just 2: Spam or Ham)
numLabels = y_tr.shape[1]

# Input data.
  
tf_train_dataset = tf.cast(tf.constant(X_train),tf.float32)
tf_train_labels = tf.constant(y_tr)
tf_test_dataset = tf.cast(tf.constant(X_test),tf.float32)
    
# Single mail input. 
tf_mail = tf.placeholder(tf.float32, shape=(1, numFeatures))
  

# Variables.
weights1 = tf.Variable(tf.truncated_normal([numFeatures, num_hidden_nodes1],stddev=np.sqrt(2.0 / (numFeatures))),name="v1")
biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]),name="v2")

weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)),name="v3")
biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]),name="v4")

weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)),name="v5")
biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]),name="v6")

weights4 = tf.Variable(tf.truncated_normal([num_hidden_nodes3, numLabels], stddev=np.sqrt(2.0 / num_hidden_nodes3)),name="v7")
biases4 = tf.Variable(tf.zeros([numLabels]),name="v8")
  
    
# Training computation.
layer1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
drop1 = tf.nn.dropout(layer1_train, keep_prob)
layer2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
drop2 = tf.nn.dropout(layer2_train, keep_prob)
layer3_train = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
drop3 = tf.nn.dropout(layer3_train, keep_prob)
logits = tf.matmul(drop3, weights4) + biases4
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
  
# Optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(loss)
  
# Predictions for the training, test data, and single mail.
    
train_prediction = tf.nn.softmax(logits)
  
layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
layer2_test = tf.nn.relu(tf.matmul(layer1_test, weights2) + biases2)
layer3_test = tf.nn.relu(tf.matmul(layer2_test, weights3) + biases3)
test_prediction = tf.nn.softmax(tf.matmul(layer3_test, weights4) + biases4)


In [16]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)


num_steps = 151
for step in range(num_steps):
    #feed={X:trainX,Y_:trainY,dropout_keep_prob:drop_train_keep}
    _, l, predictions = sess.run([optimizer, loss, train_prediction])
    acc = accuracy(predictions,y_tr)
    if (step % 10 == 0):
        print("Loss at step %d: %f" % (step, l))
        print("Accuracy: %.1f%%" % acc)


Loss at step 0: 0.702514
Accuracy: 29.5%
Loss at step 10: 0.434981
Accuracy: 87.3%
Loss at step 20: 0.386992
Accuracy: 88.3%
Loss at step 30: 0.390740
Accuracy: 88.6%
Loss at step 40: 0.368961
Accuracy: 88.7%
Loss at step 50: 0.380831
Accuracy: 88.6%
Loss at step 60: 0.377042
Accuracy: 88.9%
Loss at step 70: 0.358608
Accuracy: 88.7%
Loss at step 80: 0.346454
Accuracy: 88.9%
Loss at step 90: 0.356694
Accuracy: 88.7%
Loss at step 100: 0.342861
Accuracy: 89.0%
Loss at step 110: 0.349098
Accuracy: 89.0%
Loss at step 120: 0.365162
Accuracy: 88.8%
Loss at step 130: 0.350612
Accuracy: 88.8%
Loss at step 140: 0.342831
Accuracy: 89.2%
Loss at step 150: 0.345973
Accuracy: 89.1%


In [18]:
print("Test accuracy: %.1f%%" % accuracy(sess.run(test_prediction), y_te))

Test accuracy: 88.0%


In [10]:
cov_mat = np.cov(X_train.T)

In [13]:
tf_svd = tf.svd(cov_mat, full_matrices=False, compute_uv=True)
s_tf, u_tf, v_tf = sess.run(tf_svd)
#matrix_w=u_tf[:,0:PC_LIMIT]

In [ ]:
tf_eig = tf.self_adjoint_eig(cov_mat)
eig_vals_tf, eig_vecs_tf = sess.run(tf_eig)
print(eig_vals_tf.shape,eig_vecs_tf.shape)

In [14]:
np.savetxt("eigenvalues.csv",np.flip(np.sort(eig_vals_tf),axis=0), delimiter="\n")

(8536, 8536)